# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name:Yuqing Yan
Date: 08/17/2025

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [5]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw')
RAW.mkdir(parents=True, exist_ok=True)
load_dotenv()
print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [6]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [11]:
# --- Part 1 API Pull ---
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('ALPHAVANTAGE_API_KEY')
USE_ALPHA = bool(API_KEY)  # Determine if we should use Alpha Vantage
SYMBOL = 'AAPL'

def fetch_alpha_vantage_data(symbol, api_key):
    """Fetch stock data from Alpha Vantage API"""
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": symbol,
        "outputsize": "compact",
        "apikey": api_key,
        "datatype": "json"
    }
    
    try:
        # Make API request
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        # Check for API errors
        if "Error Message" in data:
            raise ValueError(f"API Error: {data['Error Message']}")
        if "Note" in data:  # Free API limit message
            raise ValueError(f"API Limit: {data['Note']}")
        
        # Dynamically find the time series key
        time_series_key = next((k for k in data.keys() if "Time Series" in k), None)
        if not time_series_key:
            raise ValueError("Time series data not found")
            
        # Convert to DataFrame
        df = pd.DataFrame.from_dict(data[time_series_key], orient="index")
        df = df.reset_index().rename(columns={"index": "date"})
        
        # Clean column names (convert "1. open" to "open")
        df.columns = [col.split(".")[-1].strip() for col in df.columns]
        
        df["date"] = pd.to_datetime(df["date"])
        numeric_cols = ["open", "high", "low", "close", "volume"]
        df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

        return df[['date', 'close']].rename(columns={'close': 'adj_close'})
    
    except Exception as e:
        print(f"Alpha Vantage request failed: {str(e)}")
        return None

# Fetch data from Alpha Vantage or fallback to yfinance
if USE_ALPHA:
    df_api = fetch_alpha_vantage_data(SYMBOL, API_KEY)
else:
    df_api = None

if df_api is None or df_api.empty:
    print("Using yfinance fallback...")
    try:
        import yfinance as yf
        df_api = yf.download(SYMBOL, period='3mo', progress=False).reset_index()[['Date', 'Adj Close']]
        df_api.columns = ['date', 'adj_close']
    except Exception as e:
        print(f"yfinance failed: {str(e)}")
        df_api = pd.DataFrame({"date": [pd.Timestamp.now()], "adj_close": [0]})  # Final fallback

# Validate and save data
v_api = validate(df_api, ['date', 'adj_close'])
print("API data validation:", v_api)

_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

API data validation: {'missing': [], 'shape': (100, 2), 'na_total': 0}
Saved data/raw/api_source-alpha_symbol-AAPL_20250817-133104.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [14]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
headers = {'User-Agent':'AFE-Homework/1.0'}

try:
    # Fetch Website
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()

    # Parse HTML
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Find the first table (S&P 500 constituents)
    table = soup.find('table', {'class': 'wikitable'})

    # Extract headers
    headers = [th.get_text(strip=True) for th in table.find_all('th')]

    # Extract data rows
    rows = []
    for tr in table.find_all('tr')[1:]:  # Skip header row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')])
    
    # Create DataFrame
    df_scrape = pd.DataFrame(rows, columns=headers[:len(rows[0])])
    
    # Clean column names
    df_scrape.columns = [col.replace(' ', '_').lower() for col in df_scrape.columns]

except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    # Fallback to simple table
    html = '''<table>
        <tr><th>Ticker</th><th>Company</th><th>Sector</th></tr>
        <tr><td>AAPL</td><td>Apple Inc.</td><td>Information Technology</td></tr>
        <tr><td>MSFT</td><td>Microsoft</td><td>Information Technology</td></tr>
    </table>'''

    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns))
print("Scraped data validation:", v_scrape)

Scraped data validation: {'missing': [], 'shape': (503, 8), 'na_total': 0}


In [15]:
_ = save_csv(df_scrape, prefix='scrape', site='wikipedia', table='sp500')

Saved data/raw/scrape_site-wikipedia_table-sp500_20250817-133748.csv


## Documentation

### API Source
- **Service Provider**: Alpha Vantage (primary) / Yahoo Finance (fallback)
- **Base URL**: `https://www.alphavantage.co/query`
- **Endpoint**: `TIME_SERIES_DAILY`
- **Parameters**:
  - `function=TIME_SERIES_DAILY` (daily adjusted prices)
  - `symbol=AAPL` (Apple Inc. stock)
  - `outputsize=compact` (last 100 data points)
  - `apikey=...` (loaded from .env)
- **Fallback**: yfinance `yf.download()` with 3-month daily data
- **Validation Logic**:
  - Checks for required columns: `date` and `adj_close`
  - Validates date formatting and numeric values
  - Records missing values and dataset shape

### Scrape Source
- **URL**: `https://en.wikipedia.org/wiki/List_of_S%26P_500_companies`
- **Table Description**: Current S&P 500 constituents with:
  - Ticker symbols
  - Company names
  - GICS sectors
  - Headquarters locations
- **Selector**: First table with class `wikitable`
- **Fallback**: Embedded minimal table with AAPL/MSFT examples
- **Validation Logic**:
  - Checks all scraped columns exist
  - Validates no empty rows
  - Ensures ticker symbols are present

### Assumptions & Risks
**API**:
1. Rate Limits:
   - Alpha Vantage: 5 requests/minute, 500/day (free tier)
   - yfinance: No strict limits but may throttle
2. Schema Stability:
   - Alpha Vantage JSON structure may change
   - Column naming conventions may vary
3. Data Quality:
   - Markets closed days will be missing
   - Adjusted prices may change retrospectively

**Scraping**:
1. Page Structure:
   - Wikipedia table class (`wikitable`) may change
   - Column order/headers might be modified
2. Legal:
   - Wikipedia allows scraping with proper User-Agent
   - Heavy scraping may lead to IP blocking
3. Data Consistency:
   - Company names may include special characters
   - Some cells may contain footnotes

### Environment & Reproducibility
- **`.env` Status**: Confirmed not committed (in `.gitignore`)
- **Filename Convention**:
- API: `api_source-[alpha|yfinance]_symbol-[SYMBOL]_[TIMESTAMP].csv`
- Scrape: `scrape_site-[SOURCE]_table-[NAME]_[TIMESTAMP].csv`

### Validation Summary
```python
print("API Data Validation:", v_api)
print("Scraped Data Validation:", v_scrape)